<a href="https://colab.research.google.com/github/JoseSoto9305/Colab_Notebooks/blob/main/custom_print_keras_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

In this script, we want to display the **confusion matrix** after each <br>
epoch ends during the training process.

<br>

*Maybe the versions of Python, tensorflow and numpy will change* <br> 
*when you run this script. We are using:* <br>

*python == 3.6* <br>
*numpy == 1.19*<br>
*tensorflow == 2.4.1*

---

In [ ]:
import sys
import numpy as np
import tensorflow as tf

print(f'numpy version: {np.__version__}')
print(f'tensorflow version: {tf.__version__}')
print(sys.version_info)

numpy version: 1.19.5
tensorflow version: 2.4.1
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)


In [ ]:
# First, we need some data and some model to tes  ...
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reshape data and labels
x_train = x_train.reshape(-1, 784).astype(np.float32) / 255.0
x_test = x_test.reshape(-1, 784).astype(np.float32) / 255.0

# We will display a confusion maxtrix, so we need the labels as hot vectors
n_labels = np.unique(y_train).size

y_train = tf.keras.utils.to_categorical(y_train, num_classes=n_labels)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=n_labels)

print(f'Data shape: {x_train.shape}')
print(f'Labels shape: {y_train.shape}')

Data shape: (60000, 784)
Labels shape: (60000, 10)


In [ ]:
# Build some model ...
model = tf.keras.Sequential([tf.keras.layers.Dense(n_labels, 
                                                   input_dim=x_train.shape[-1])])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Build our custom metric. In this case we will build a confusion matrix
# that we want to display after each epoch ends.

class ConfusionMatrix(tf.keras.metrics.Metric):

    def __init__(self, name='cfs_mtx', n_labels=4, **kwargs):
        
        super(ConfusionMatrix, self).__init__(name=name, **kwargs)
        self.n_labels = n_labels

        # This will allocate our confusion matrix
        self.mtx = self.add_weight(shape=(self.n_labels, self.n_labels), 
                                   name='mtx', 
                                   initializer='zeros', 
                                   dtype=tf.int32)


    def update_state(self, y_true, y_pred, sample_weight=None):
        # Compute metric
        y_true = tf.cast(tf.argmax(y_true, axis=-1), dtype=tf.int32)
        y_pred = tf.cast(tf.argmax(y_pred, axis=-1), dtype=tf.int32)
        mtx = tf.math.confusion_matrix(y_true, y_pred, num_classes=self.n_labels,
                                       dtype=tf.int32)
        if sample_weight is not None:
            # Do something
            pass
        self.mtx.assign_add(mtx)


    def result(self):
        return self.mtx


    def reset_states(self):
        # Set to zero when epoch ends
        self.mtx.assign(tf.constant(0, shape=[self.n_labels, self.n_labels], dtype=tf.int32))

In [ ]:
# Build our custom callback to print the training process
class CustomPrinted(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs):

        print(f'\nEnd of epoch: {epoch+1}\n')
        print('====================================================')
        print(f'loss: {logs["loss"]:.2f}   val_loss: {logs["val_loss"]:.2f}')
        print('====================================================')

        print('\nValidation Confusion Matrix:\n')
        print(f'{logs["val_cfs_mtx"]}\n')
        print('====================================================\n')

In [ ]:
# Compile the model with our metric
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.1),
              loss='mean_squared_error',
              metrics=[ConfusionMatrix(n_labels=n_labels)])



In [ ]:
# Set our custom callback
callbacks = [CustomPrinted()]

hist = model.fit(x_train, y_train,
                 batch_size=16,
                 steps_per_epoch=10,
                 epochs=5,
                 verbose=1,
                 validation_split=0.5,
                 callbacks=[callbacks])


Epoch 1/5
10/10 [==============================] - 3s 222ms/step - loss: 171.0591 - cfs_mtx: 0.9455 - val_loss: 6.1958 - val_cfs_mtx: 300.0000

End of epoch: 1

loss: 102.64   val_loss: 6.20

Validation Confusion Matrix:

[[   0  147  106  353   37    2 1972  338    6    1]
 [   0    2   10   25    0    0 2818   32  432    0]
 [   0  179  221  100   47    6 1701  711   22   23]
 [   0   50  201 1407    7    2 1079  250   58    4]
 [   0  143   19  135   91    9 2140   25  328   26]
 [   1   66  132  485    3   20 1571  287  146    1]
 [   0    7   21  197    1   15 2301  220  181    0]
 [   0  265  252  172    0    3 2183   88  147   48]
 [   0   10   37  109    0    0 2743   54   23    0]
 [   0  109   40  311    2    0 1905   26  503   50]]


Epoch 2/5
10/10 [==============================] - 2s 167ms/step - loss: 6.6361 - cfs_mtx: 0.9455 - val_loss: 27.1700 - val_cfs_mtx: 300.0000

End of epoch: 2

loss: 10.40   val_loss: 27.17

Validation Confusion Matrix:

[[   0 1232  344   26   

Great !!!!!!!!!!!!! :D


But notice that it prints our **cft_mtx** as a float value after <br> 
each training batch ends. We can disable this modifying the<br>
**on_train_batch_end** method and setting verbose to zero:

---

In [ ]:
# Build our custom callback to print the training process
class CustomPrinted(tf.keras.callbacks.Callback):

    def on_epoch_begin(self, epoch, logs=None):
        print(f'\nEpoch: {epoch+1}\n')

    def on_train_batch_end(self, batch, logs=None):
        print(f'training batch: {batch} ... loss: {logs["loss"]:.2f}')

    def on_epoch_end(self, epoch, logs):

        print(f'\nEnd of epoch: {epoch+1}\n')
        print('====================================================')
        print(f'loss: {logs["loss"]:.2f}   val_loss: {logs["val_loss"]:.2f}')
        print('====================================================')

        print('\nValidation Confusion Matrix:\n')
        print(f'{logs["val_cfs_mtx"]}\n')
        print('====================================================\n')

In [ ]:
# Compile the model with our metric
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.1),
              loss='mean_squared_error',
              metrics=[ConfusionMatrix(n_labels=n_labels)])


In [ ]:
# Set our custom callback
callbacks = [CustomPrinted()]

hist = model.fit(x_train, y_train,
                 batch_size=16,
                 steps_per_epoch=10,
                 epochs=5,
                 verbose=0,
                 validation_split=0.5,
                 callbacks=[callbacks])


Epoch: 1

training batch: 0 ... loss: 5.91
training batch: 1 ... loss: 447.01
training batch: 2 ... loss: 308.16
training batch: 3 ... loss: 235.90
training batch: 4 ... loss: 191.18
training batch: 5 ... loss: 160.37
training batch: 6 ... loss: 138.71
training batch: 7 ... loss: 122.50
training batch: 8 ... loss: 109.34
training batch: 9 ... loss: 99.17

End of epoch: 1

loss: 99.17   val_loss: 9.79

Validation Confusion Matrix:

[[  60   18    0    5    5   22   29   86 2737    0]
 [   0   25    0    2    0    2   21   22 3247    0]
 [  43   19    4    5    2  125   58  280 2469    5]
 [  37    1    1   23    4  145   19  178 2641    9]
 [  11   22    1    4   14   95   17   93 2643   16]
 [  66    8    4    5   15  118   65  126 2295   10]
 [  47   16    2    7    1   90  117   81 2580    2]
 [  18   71   21   19   18   68   66  777 2088   12]
 [  24    0    1    0    5   54   18   58 2810    6]
 [  11   53    1   16   17   97   78  229 2431   13]]



Epoch: 2

training batch: 0 ..

---

This is just an example, so you can build a more sofisticate summary <br>
I recommend you read this [tutorial of keras of how to write callbacks]((https://keras.io/guides/writing_your_own_callbacks/)). <br>
And also, this [tutorial
of how to write custom Metrics](https://keras.io/api/metrics/) . <br>

Best

;) 

---